In [1]:
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
import tensorflow as tf
from tensorflow import keras

2023-12-05 02:13:18.522024: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [3]:
from tensorflow.keras.preprocessing.image import load_img

In [4]:
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.applications.xception import preprocess_input
from tensorflow.keras.applications.xception import decode_predictions

In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [6]:
train_gen = ImageDataGenerator(rescale=1./255)

train_ds = train_gen.flow_from_directory(
    './data/train',
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary',
    shuffle=True
)

Found 3677 images belonging to 2 classes.


In [7]:
train_ds.class_indices

{'bee': 0, 'wasp': 1}

In [8]:
val_gen = ImageDataGenerator(rescale=1./255)

val_ds = val_gen.flow_from_directory(
    './data/test',
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary',
    shuffle=True
)

Found 918 images belonging to 2 classes.


In [9]:
inputs = keras.Input(shape=(150, 150, 3))
conv = keras.layers.Conv2D(filters=32, kernel_size=(3,3), activation='relu')(inputs)
pool = keras.layers.MaxPooling2D(pool_size=(2, 2))(conv)
vector = keras.layers.Flatten()(pool)
inner = keras.layers.Dense(64, activation='relu')(vector)
outputs = keras.layers.Dense(1, activation='sigmoid')(inner)
model = keras.Model(inputs, outputs)
    

2023-12-05 02:13:24.835871: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-12-05 02:13:24.842904: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-12-05 02:13:24.843500: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-12-05 02:13:24.844434: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the approp

In [10]:
optimizer = keras.optimizers.SGD(learning_rate=0.002, momentum=0.8)
    
loss = keras.losses.BinaryCrossentropy()
    
model.compile(
    optimizer=optimizer,
    loss=loss,
    metrics=['accuracy']
)

In [11]:
history = model.fit(train_ds, epochs=10, validation_data=val_ds)

Epoch 1/10


2023-12-05 02:13:26.990610: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8100
2023-12-05 02:13:27.639853: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-12-05 02:13:27.640318: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-12-05 02:13:27.640358: W tensorflow/stream_executor/gpu/asm_compiler.cc:80] Couldn't get ptxas version string: INTERNAL: Couldn't invoke ptxas --version
2023-12-05 02:13:27.640840: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-12-05 02:13:27.640927: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] INTERNAL: Failed to launch ptxas
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.


184/184 [==============================] - 14s 61ms/step - loss: 0.6722 - accuracy: 0.5681 - val_loss: 0.6458 - val_accuracy: 0.5468
Epoch 2/10
184/184 [==============================] - 8s 45ms/step - loss: 0.6319 - accuracy: 0.6217 - val_loss: 0.5874 - val_accuracy: 0.6961
Epoch 3/10
184/184 [==============================] - 8s 45ms/step - loss: 0.5838 - accuracy: 0.6921 - val_loss: 0.5556 - val_accuracy: 0.7407
Epoch 4/10
184/184 [==============================] - 8s 45ms/step - loss: 0.5462 - accuracy: 0.7329 - val_loss: 0.5537 - val_accuracy: 0.7407
Epoch 5/10
184/184 [==============================] - 8s 45ms/step - loss: 0.5202 - accuracy: 0.7601 - val_loss: 0.5378 - val_accuracy: 0.7593
Epoch 6/10
184/184 [==============================] - 8s 45ms/step - loss: 0.4990 - accuracy: 0.7735 - val_loss: 0.5486 - val_accuracy: 0.7244
Epoch 7/10
184/184 [==============================] - 8s 45ms/step - loss: 0.4662 - accuracy: 0.7925 - val_loss: 0.5448 - val_accuracy: 0.7331
Epoch 8/1

In [12]:
model.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 74, 74, 32)       0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 175232)            0         
                                                                 
 dense (Dense)               (None, 64)                11214912  
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                             

In [13]:
acc = history.history['accuracy']
acc_median = np.median(acc)
acc_median

0.7667935788631439

In [14]:
loss=history.history['loss']
loss_std = np.std(loss)
loss_std

0.08838078047348406

In [15]:
train_gen_data_augmentation = ImageDataGenerator(rescale=1./255,
                                                 rotation_range=50,
                                                 width_shift_range=0.1,
                                                 height_shift_range=0.1,
                                                 zoom_range=0.1,
                                                 horizontal_flip=True,
                                                 fill_mode='nearest')

train_ds_da = train_gen_data_augmentation.flow_from_directory(
    './data/train',
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary',
    shuffle=True
)

Found 3677 images belonging to 2 classes.


In [16]:
test_gen = ImageDataGenerator(rescale=1./255)

test_ds = test_gen.flow_from_directory(
    './data/test',
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary',
    shuffle=True
)

Found 918 images belonging to 2 classes.


In [17]:
history = model.fit(
    train_ds_da,
    epochs=10,
    validation_data=test_ds
)

Epoch 1/10
184/184 [==============================] - 23s 125ms/step - loss: 0.5060 - accuracy: 0.7561 - val_loss: 0.4753 - val_accuracy: 0.7800
Epoch 2/10
184/184 [==============================] - 23s 125ms/step - loss: 0.4870 - accuracy: 0.7756 - val_loss: 0.4627 - val_accuracy: 0.7898
Epoch 3/10
184/184 [==============================] - 23s 126ms/step - loss: 0.4862 - accuracy: 0.7764 - val_loss: 0.4710 - val_accuracy: 0.7963
Epoch 4/10
184/184 [==============================] - 23s 125ms/step - loss: 0.4826 - accuracy: 0.7792 - val_loss: 0.5108 - val_accuracy: 0.7462
Epoch 5/10
184/184 [==============================] - 23s 125ms/step - loss: 0.4779 - accuracy: 0.7775 - val_loss: 0.4888 - val_accuracy: 0.7756
Epoch 6/10
184/184 [==============================] - 23s 124ms/step - loss: 0.4626 - accuracy: 0.7868 - val_loss: 0.4661 - val_accuracy: 0.7810
Epoch 7/10
184/184 [==============================] - 23s 126ms/step - loss: 0.4662 - accuracy: 0.7906 - val_loss: 0.4887 - val_ac

In [18]:
test_loss = history.history['val_loss']
loss_mean= np.mean(test_loss)
loss_mean

0.47471369206905367

In [20]:
acc_test_mean_aug = np.mean(history.history['val_accuracy'][5:10])
acc_test_mean_aug


0.784749448299408